In [1]:
# -------------------------
# 0) CONFIG (edit these)
# -------------------------
import os, sys


# Project code (must contain the csiro package)
CSIRO_CODE_DIR = "/notebooks/CSIRO"

# DINOv3 repo/code dir (must contain hubconf.py)
DINO_REPO = "/notebooks/dinov3"

# Ensemble checkpoints (.pt)
WEIGHTS_PATHS = [
    "/notebooks/kaggle/csiro/output/a_v6_cv_paramsmodegkfcv_seed126015n_splits5n_models1tau_050.pt",
    "/notebooks/kaggle/csiro/output/a_v6_cv_paramsmodegkfcv_seed1527n_splits5n_models2tau_p_f72.pt",
]

# Competition data
COMP_ROOT = "/notebooks/kaggle/csiro"
TEST_CSV = f"{COMP_ROOT}/train.csv"
# IMPORTANT: test.csv image_path values look like "test/IDxxxx.jpg", so IMAGE_ROOT should be COMP_ROOT
IMAGE_ROOT = COMP_ROOT
SYMLINK = False
# Inference params
IMG_SIZE = 512
BATCH_SIZE = 64
NUM_WORKERS = os.cpu_count() - 3
DEVICE = "cuda"  # or "cpu"

# TTA / ensemble knobs
TTA_N = 4
TTA_BCS = 0.0
TTA_HUE = 0.0
TTA_AGG = "mean"
INNER_AGG = "mean"
OUTER_AGG = "mean"  # aggregate over CV -> aggregate over seeds

sub_id = "2"
OUTPUT_PATH = f"/notebooks/kaggle/csiro/sub/sub{sub_id}.csv"
WRITE_SUB = True

sys.path.insert(0, "/notebooks/CSIRO")
os.environ["COMET_API_KEY"]="R7OuT6FolA02VmQRI82xDN48O"
os.environ["COMET_DISABLE_AUTO_LOGGING"]="1"
os.environ["KAGGLE_API_TOKEN"]="KGAT_4aac4afa024236c6a163b5e4af00b4c7"
os.environ["DINO_WB"]="https://dinov3.llamameta.net/dinov3_vitb16/dinov3_vitb16_pretrain_lvd1689m-73cec8be.pth?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoidW84aXJvdGQyeThwcGpuNXFveGthZTE4IiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZGlub3YzLmxsYW1hbWV0YS5uZXRcLyoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NjU5NzI4MTd9fX1dfQ__&Signature=H5H5kLVc6V83i-s2euNHx6t9KlVeG27QKX6qtkXNiLwEzuCshJD4RfwUbQv8oBJOZXPezAVJZPRkYRdsb4jh-LQ72DZtEuNkjNKHf7Pn57wzee0bjEYjWdJmOqK4waaSe9TQqELM%7EPgzdAT4LCSHYcFQ%7EleRnHGGGJiHBmTd6e1xZYhvUCfkvVD1TG-zM7R0-P%7EMLetHMvWl%7EUapCMYthsWqZctsYAQKUQxsLrly8Y4EaM8hm5nowpArPZC4myNO1iiXld5Hc3t9CVLEdYT7LIct0x6cf3-B-6WOgxGb7LdLPCcZPPfoGgX3KGtTAgNQYOpGFs-hgILFHRKVOJ7T3A__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=1893388161261111"
os.environ["DINO_WL"]="https://dinov3.llamameta.net/dinov3_vitl16/dinov3_vitl16_pretrain_lvd1689m-8aa4cbdd.pth?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoidW84aXJvdGQyeThwcGpuNXFveGthZTE4IiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZGlub3YzLmxsYW1hbWV0YS5uZXRcLyoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NjU5NzI4MTd9fX1dfQ__&Signature=H5H5kLVc6V83i-s2euNHx6t9KlVeG27QKX6qtkXNiLwEzuCshJD4RfwUbQv8oBJOZXPezAVJZPRkYRdsb4jh-LQ72DZtEuNkjNKHf7Pn57wzee0bjEYjWdJmOqK4waaSe9TQqELM%7EPgzdAT4LCSHYcFQ%7EleRnHGGGJiHBmTd6e1xZYhvUCfkvVD1TG-zM7R0-P%7EMLetHMvWl%7EUapCMYthsWqZctsYAQKUQxsLrly8Y4EaM8hm5nowpArPZC4myNO1iiXld5Hc3t9CVLEdYT7LIct0x6cf3-B-6WOgxGb7LdLPCcZPPfoGgX3KGtTAgNQYOpGFs-hgILFHRKVOJ7T3A__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=1893388161261111"
os.environ["DINO_WL_plus"]="https://dinov3.llamameta.net/dinov3_vith16plus/dinov3_vith16plus_pretrain_lvd1689m-7c1da9a5.pth?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoidW84aXJvdGQyeThwcGpuNXFveGthZTE4IiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZGlub3YzLmxsYW1hbWV0YS5uZXRcLyoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NjU5NzI4MTd9fX1dfQ__&Signature=H5H5kLVc6V83i-s2euNHx6t9KlVeG27QKX6qtkXNiLwEzuCshJD4RfwUbQv8oBJOZXPezAVJZPRkYRdsb4jh-LQ72DZtEuNkjNKHf7Pn57wzee0bjEYjWdJmOqK4waaSe9TQqELM%7EPgzdAT4LCSHYcFQ%7EleRnHGGGJiHBmTd6e1xZYhvUCfkvVD1TG-zM7R0-P%7EMLetHMvWl%7EUapCMYthsWqZctsYAQKUQxsLrly8Y4EaM8hm5nowpArPZC4myNO1iiXld5Hc3t9CVLEdYT7LIct0x6cf3-B-6WOgxGb7LdLPCcZPPfoGgX3KGtTAgNQYOpGFs-hgILFHRKVOJ7T3A__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=1893388161261111"
os.environ["DEFAULT_DINO_REPO_DIR"]="/notebooks/dinov3"
os.environ["DEFAULT_DATA_ROOT"]="/notebooks/kaggle/csiro"
os.environ["DINO_WEIGHTS_PATH"] = DINO_WEIGHTS = "/notebooks/kaggle/csiro/weights/dinov3/dinov3_vitb16_pretrain.pth"

# --- Basic guards (set to None if unknown and fill later) ---
for name, val in {
    "CSIRO_CODE_DIR": CSIRO_CODE_DIR,
    "DINO_REPO": DINO_REPO,
    "DINO_WEIGHTS": DINO_WEIGHTS,
    "WEIGHTS_PATHS": WEIGHTS_PATHS,
    "TEST_CSV": TEST_CSV,
    "IMAGE_ROOT": IMAGE_ROOT,
}.items():
    if val is None:
        raise ValueError(f"{name} is None; set it before running.")

sys.path.insert(0, CSIRO_CODE_DIR)


# -------------------------
# 1) Imports
# -------------------------
import pandas as pd
import numpy as np
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset
from PIL import Image

import csiro
from csiro.config import TARGETS, dino_hub_name, DEFAULT_MODEL_SIZE, DEFAULT_PLUS
from csiro.transforms import PadToSquare
from csiro.data import TTADataset
from csiro.eval import predict_ensemble, load_ensemble_states
print("TARGETS:", TARGETS)

if SYMLINK:
    from pathlib import Path
    # Create checkpoints dir
    ckpt_dir = Path(torch.hub.get_dir()) / "checkpoints"
    ckpt_dir.mkdir(parents=True, exist_ok=True)

    # Symlink the weight file from /kaggle/input (read-only) into the hub cache
    src = Path(DINO_WEIGHTS)                  # /kaggle/input/.../dinov3_vitb16_pretrain.pth
    dst = ckpt_dir / src.name                 # /kaggle/working/torch_hub/checkpoints/dinov3_vitb16_pretrain.pth

    if not dst.exists():
        dst.symlink_to(src)
        
# -------------------------
# 2) Load checkpoint + backbone
# -------------------------
states = load_ensemble_states(WEIGHTS_PATHS)
sys.path.insert(0, DINO_REPO)
backbone = torch.hub.load(
    DINO_REPO,
    dino_hub_name(model_size=str(DEFAULT_MODEL_SIZE), plus=str(DEFAULT_PLUS)),
    source="local",
    weights=os.environ["DINO_WEIGHTS_PATH"],
).cuda()
backbone.eval()
print("Loaded states", type(states))


# -------------------------
# 3) Read test.csv (long format)
# -------------------------
df = pd.read_csv(TEST_CSV)
print("test.csv columns:", list(df.columns))
print(df.head(3))

IMAGE_PATH_COL = "image_path"
TARGET_NAME_COL = "target_name"
SAMPLE_ID_COL = "sample_id"

if IMAGE_PATH_COL not in df.columns:
    raise ValueError(f"Expected column {IMAGE_PATH_COL} in test.csv")
if TARGET_NAME_COL not in df.columns:
    raise ValueError(f"Expected column {TARGET_NAME_COL} in test.csv")

df_img = df.drop_duplicates(subset=[IMAGE_PATH_COL]).reset_index(drop=True)
print("rows (long):", len(df), "unique images:", len(df_img))

TARGETS: ['Dry_Green_g', 'Dry_Clover_g', 'Dry_Dead_g', 'GDM_g', 'Dry_Total_g']
Loaded states <class 'list'>
test.csv columns: ['sample_id', 'image_path', 'Sampling_Date', 'State', 'Species', 'Pre_GSHH_NDVI', 'Height_Ave_cm', 'target_name', 'target']
                    sample_id              image_path Sampling_Date State  \
0  ID1011485656__Dry_Clover_g  train/ID1011485656.jpg      2015/9/4   Tas   
1    ID1011485656__Dry_Dead_g  train/ID1011485656.jpg      2015/9/4   Tas   
2   ID1011485656__Dry_Green_g  train/ID1011485656.jpg      2015/9/4   Tas   

           Species  Pre_GSHH_NDVI  Height_Ave_cm   target_name   target  
0  Ryegrass_Clover           0.62         4.6667  Dry_Clover_g   0.0000  
1  Ryegrass_Clover           0.62         4.6667    Dry_Dead_g  31.9984  
2  Ryegrass_Clover           0.62         4.6667   Dry_Green_g  16.2751  
rows (long): 1785 unique images: 357


In [3]:
# -------------------------
# 4) Dataset + inference
# -------------------------
class TestDataset(Dataset):
    def __init__(self, df, root, img_col, img_size):
        self.df = df.reset_index(drop=True)
        self.root = root
        self.img_col = img_col
        self.pre = T.Compose([
            T.Lambda(lambda im: im.convert("RGB")),
            PadToSquare(fill=0),
            T.Resize((int(img_size), int(img_size)), antialias=True),
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        rel = self.df.loc[i, self.img_col]
        p = os.path.join(self.root, rel) if self.root else rel
        with Image.open(p) as im:
            x = self.pre(im)
        return x

ds_base = TestDataset(df_img, IMAGE_ROOT, IMAGE_PATH_COL, IMG_SIZE)

ds = TTADataset(
    ds_base,
    tta_n=TTA_N,
    bcs_val=TTA_BCS,
    hue_val=TTA_HUE,
)
preds = predict_ensemble(
    ds,
    states=states,
    backbone=backbone,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    device=DEVICE,
    tta_agg=TTA_AGG,
    inner_agg=INNER_AGG,
    outer_agg=OUTER_AGG,
)

preds = preds.detach().cpu()
print("preds shape:", tuple(preds.shape))

# -------------------------
# 5) Build submission (long format)
# -------------------------
target_to_idx = {t: i for i, t in enumerate(TARGETS)}
preds_np = preds.numpy()

pred_by_path = {
    df_img.loc[i, IMAGE_PATH_COL]: preds_np[i]
    for i in range(len(df_img))
}

if SAMPLE_ID_COL in df.columns:
    sample_ids = df[SAMPLE_ID_COL].astype(str)
else:
    image_ids = df[IMAGE_PATH_COL].apply(lambda p: os.path.splitext(os.path.basename(p))[0])
    sample_ids = image_ids + "__" + df[TARGET_NAME_COL].astype(str)

targets = []
for i, row in df.iterrows():
    p = row[IMAGE_PATH_COL]
    t_name = row[TARGET_NAME_COL]
    if t_name not in target_to_idx:
        raise ValueError(f"Unknown target_name: {t_name}")
    vec = pred_by_path[p]
    targets.append(float(vec[target_to_idx[t_name]]))

sub = pd.DataFrame({
    "sample_id": sample_ids.values,
    "target": np.asarray(targets, dtype=np.float32),
})

assert len(sub) == len(df)
assert list(sub.columns) == ["sample_id", "target"]
assert np.isfinite(sub["target"].values).all()


if WRITE_SUB:
    sub.to_csv(OUTPUT_PATH, index=False)
    print("Wrote - ", OUTPUT_PATH)

print(sub.head(10))


preds shape: (5, 5)


IndexError: index 5 is out of bounds for axis 0 with size 5

In [4]:
df_img

,sample_id,image_path,Sampling_Date,State,Species,Pre_GSHH_NDVI,Height_Ave_cm,target_name,target
0,ID1011485656__Dry_Clover_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Clover_g,0.0000
1,ID1012260530__Dry_Clover_g,train/ID1012260530.jpg,2015/4/1,NSW,Lucerne,0.55,16.0000,Dry_Clover_g,0.0000
2,ID1025234388__Dry_Clover_g,train/ID1025234388.jpg,2015/9/1,WA,SubcloverDalkeith,0.38,1.0000,Dry_Clover_g,6.0500
3,ID1028611175__Dry_Clover_g,train/ID1028611175.jpg,2015/5/18,Tas,Ryegrass,0.66,5.0000,Dry_Clover_g,0.0000
4,ID1035947949__Dry_Clover_g,train/ID1035947949.jpg,2015/9/11,Tas,Ryegrass,0.54,3.5000,Dry_Clover_g,0.4343
...,...,...,...,...,...,...,...,...,...
352,ID975115267__Dry_Clover_g,train/ID975115267.jpg,2015/7/8,WA,Clover,0.73,3.0000,Dry_Clover_g,40.0300
353,ID978026131__Dry_Clover_g,train/ID978026131.jpg,2015/9/4,Tas,Clover,0.83,3.1667,Dry_Clover_g,24.6445
354,ID980538882__Dry_Clover_g,train/ID980538882.jpg,2015/2/24,NSW,Phalaris,0.69,29.0000,Dry_Clover_g,0.0000
355,ID980878870__Dry_Clover_g,train/ID980878870.jpg,2015/7/8,WA,Clover,0.74,2.0000,Dry_Clover_g,32.3575
